# Swarm Maximum

In [25]:
class Point2d:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    # Subtract two points to get a vector start_point --> self.
    def __sub__(self, start_point):
        return Vector2d(self.x - start_point.x, self.y - start_point.y)

    # Add this point to a vector to get a new point
    # (Or add the coordinates of two points, e.g. to calculate an average.)
    def __add__(self, other):
        return Point2d(self.x + other.x, self.y + other.y)

    # Scaling isn't really a point operation.
    # It's used by this program to calculate averages.
    # Scale this point by multiplying.
    def __mul__(self, scale):
        return Point2d(self.x * scale, self.y * scale)

    # Scale this point by dividing.
    def __truediv__(self, scale):
        return self * (1.0 / scale)

    def __str__(self):
        return f'({self.x}, {self.y})'

    def draw(self, canvas, color):
        RADIUS = 3
        canvas.create_oval(
            self.x - RADIUS, self.y - RADIUS,
            self.x + RADIUS, self.y + RADIUS,
            fill=color, outline=color)

In [26]:
import math

class Vector2d:
    # Initialize from either (a, b) coordinates or two points a --> b.
    def __init__(self, a, b):
        if type(a) is Point2d:
            # Initialize from two points a --> b.
            self.x = b.x - a.x
            self.y = b.y - a.y
        else:
            # Initialize from (a, b) coordinates.
            self.x = a
            self.y = b

    # Add two vectors to get a new vector or
    # add the vector to a point to get a new point.
    def __add__(self, other):
        if type(other) is Vector2d:
            # Add two vectors to get a new vector.
            return Vector2d(self.x + other.x, self.y + other.y)
        else:
            # Add the vector to a point to get a new point.
            return Point2d(self.x + other.x, self.y + other.y)

    # Return the negation of this vector.
    def __neg__(self):
        return Vector2d(-self.x, -self.y)

    # Subtract two vectors.
    def __sub__(self, other):
        return self + -other

    # Scale this vector by multiplying.
    def __mul__(self, scale):
        return Vector2d(self.x * scale, self.y * scale)

    # Scale this vector by dividing.
    def __truediv__(self, scale):
        return self * (1.0 / scale)

    # Scale the vector.
    # Return self so we can use it in further calculations.
    def scale(self, scale):
        self.x *= scale
        self.y *= scale
        return self

    # Return the vector's length.
    def length(self):
        return math.sqrt(self.x * self.x + self.y * self.y)

    # Set the vector's length to new_length.
    # Return self so we can use it in further calculations.
    def set_length(self, new_length):
        old_length = self.length()
        if old_length < 0.01: return self # Don't divide by zero.
        self.x *= new_length / old_length
        self.y *= new_length / old_length
        return self

    # Set the vector's length to 1.
    # Return self so we can use it in further calculations.
    def normalize(self):
        return self.set_length(1)

    def __str__(self):
        return f'<{self.x}, {self.y}>'

    def draw(self, canvas, start_point, color):
        end_point = self + start_point
        canvas.create_line(
            start_point.x, start_point.y,
            end_point.x, end_point.y,
            fill=color)

In [27]:
import math

# Return the function's value F(x, y).
def F(point):
    r2 = (point.x * point.x + point.y * point.y) / 4
    r = math.sqrt(r2)

    theta = math.atan2(point.y, point.x)
    z = 3 * math.exp(-r2) * \
            math.sin(2 * math.pi * r) * \
            math.cos(3 * theta)
    return z

In [28]:
import random

class Bug:
    def __init__(self, f, location, velocity):
        self.f = f
        self.location = location
        self.velocity = velocity
        self.best_location = location
        self.best_value = f(location)

        self.is_active = True
        self.moves_since_changed = 0

    # Move the bug.
    def move(self, elapsed):
        if not self.is_active:
            return
        self.inertia_velocity = self.velocity * Bug.inertia_weight * elapsed
        self.cognitive_velocity = (self.best_location - self.location).normalize() * Bug.cognitive_weight * random.uniform(0, 1) * elapsed
        self.social_velocity = (Bug.global_best_location - self.location).normalize() * Bug.social_weight * random.uniform(0, 1) * elapsed
        self.velocity = self.inertia_velocity + self.cognitive_velocity + self.social_velocity
        if self.velocity.length() > Bug.max_speed:
            self.velocity.set_length(Bug.max_speed)
        self.location = self.location + self.velocity * elapsed
        if self.value() > self.best_value:
            self.best_value = self.value()
            self.best_location = self.location
            if self.value() > Bug.global_best_value:
                Bug.global_best_value = self.value()
                Bug.global_best_location = self.location
            else:
                self.moves_since_changed+=1
                if self.moves_since_changed >= Bug.lock_after:
                    self.is_active = False
    
    def value(self):
        """ Return the bug's current value."""
        return self.f(self.location)

In [29]:
import tkinter as tk

# Get the text in an Entry widget and
# convert it to an int.
def get_int(entry):
    return int(entry.get())

# Get the text in an Entry widget and
# convert it to a float.
def get_float(entry):
    return float(entry.get())

# Make Label and Entry widgets for a field.
# Return the Entry widget.
def make_field(parent, label_width, label_text, entry_width, entry_default):
    frame = tk.Frame(parent)
    frame.pack(side=tk.TOP)

    label = tk.Label(frame, text=label_text, width=label_width, anchor=tk.W)
    label.pack(side=tk.LEFT)

    entry = tk.Entry(frame, width=entry_width, justify='right')
    entry.insert(tk.END, entry_default)
    entry.pack(side=tk.LEFT)

    return entry

In [30]:
import tkinter as tk
import time

MARGIN = 5
WINDOW_WID = 485
WINDOW_HGT = 320
CANVAS_HGT = WINDOW_HGT - 2 * MARGIN
CANVAS_WID = CANVAS_HGT
TICK_MS = 20 # 20 milliseconds between ticks.

class App:
    def kill_callback(self):
        self.window.destroy()

    def __init__(self):
        self.running = False

        # Make the main interface.
        self.window = tk.Tk()
        self.window.title('swarm_minimum')
        self.window.protocol('WM_DELETE_WINDOW', self.kill_callback)
        self.window.geometry(f'{WINDOW_WID}x{WINDOW_HGT}')

        # Build the rest of the UI.
        self.build_ui()

        # Display the window.
        self.window.focus_force()
        self.window.mainloop()

    def build_ui(self):
        self.canvas = tk.Canvas(self.window, bg='white',
            borderwidth=1, highlightthickness=0, width=CANVAS_WID, height=CANVAS_HGT)
        self.canvas.pack(side=tk.LEFT, padx=MARGIN, pady=MARGIN, anchor=tk.NW)

        # Make a frame to hold labels, text boxes, and buttons.
        right_frame = tk.Frame(self.window)
        right_frame.pack(side=tk.LEFT, padx=MARGIN, pady=MARGIN, anchor=tk.NW)

        # Number of bugs, cognition acceleration, and social acceleration.
        LABEL_WID = 15
        ENTRY_WID = 5
        self.num_bugs_entry = make_field(right_frame, LABEL_WID, '# Bugs:', ENTRY_WID, 100)
        self.inertia_weight_entry = make_field(right_frame, LABEL_WID, 'Inertia Weight:', ENTRY_WID, 100)
        self.cognition_weight_entry = make_field(right_frame, LABEL_WID, 'Cognition Weight:', ENTRY_WID, 100)
        self.social_weight_entry = make_field(right_frame, LABEL_WID, 'Social Weight:', ENTRY_WID, 100)

        # Make a vertical gap.
        gap = tk.Frame(right_frame, width=1, height=10)
        gap.pack(side=tk.TOP)

        # Max speed and lock after.
        self.max_speed_entry = make_field(right_frame, LABEL_WID, 'Max Speed:', ENTRY_WID, 1)
        self.lock_after_entry = make_field(right_frame, LABEL_WID, 'Lock After:', ENTRY_WID, 100)

        # Make a vertical gap.
        gap = tk.Frame(right_frame, width=1, height=10)
        gap.pack(side=tk.TOP)

        # Run button.
        self.run_button = tk.Button(right_frame, text='Run', width=7, command=self.run)
        self.run_button.pack(side=tk.TOP)

        # Make a vertical gap.
        gap = tk.Frame(right_frame, width=1, height=10)
        gap.pack(side=tk.TOP)

        # Make the result label.
        frame = tk.Frame(right_frame)
        frame.pack(side=tk.TOP)
        label = tk.Label(frame, text='Minimum:', width=8, anchor=tk.W)
        label.pack(side=tk.LEFT)
        self.minimum_label = tk.Label(frame, text='', width=12, anchor=tk.W)
        self.minimum_label.pack(side=tk.LEFT)

    # Start or stop the simulation.
    def run(self):
        if self.run_button['text'] == 'Run':
            self.start()
        else:
            self.stop()

    # Make some bugs and start the simulation.
    def start(self):
        # World coordinates.
        self.wxmin = -3.5
        self.wxmax = 3.5
        self.wymin = -3.5
        self.wymax = 3.5

        # Device coordinates.
        self.dxmin = 0
        self.dxmax = self.canvas.winfo_width()
        self.dymin = 0
        self.dymax = self.canvas.winfo_width()
        self.xscale = (self.dxmax - self.dxmin) / (self.wxmax - self.wxmin)
        self.yscale = (self.dymax - self.dymin) / (self.wymax - self.wymin)

        # Set the Bug class properties.
        Bug.cognitive_weight = get_float(self.cognition_weight_entry)
        Bug.social_weight = get_float(self.social_weight_entry)
        Bug.inertia_weight = get_float(self.inertia_weight_entry)
        Bug.max_speed = get_float(self.max_speed_entry)
        Bug.lock_after = get_int(self.lock_after_entry)

        # Make the bugs.
        num_bugs = get_int(self.num_bugs_entry)
        self.bugs = []
        for i in range(num_bugs):
            location = Point2d(self.wxmin + (self.wxmax - self.wxmin) * random.uniform(0, 1), 
                               self.wymin + (self.wymax - self.wymin) * random.uniform(0, 1))
            velocity = Vector2d(-1 + 2 * random.uniform(0, 1), -1 + 2 * random.uniform(0, 1))
            self.bugs.append(Bug(F, location, velocity))

        # Set the initial best point.
        Bug.global_best_location = self.bugs[0].location
        Bug.global_best_value = self.bugs[0].best_value

        # Start the timer.
        self.run_button['text'] = 'Stop'
        self.last_time = time.time()
        self.running = True
        self.tick()

    def stop(self):
        self.running = False
        self.run_button['text'] = 'Run'

    # Draw a frame of the simulation.
    def tick(self):
        if not self.running:
            return

        # Get the elapsed time in seconds.
        now = time.time()
        elapsed = now - self.last_time
        self.last_time = now

        # Move the bugs.
        for bug in self.bugs:
            bug.move(elapsed)

        # Display the current global minimum.
        self.minimum_label['text'] = f'{Bug.global_best_value:.12f}'

        # Redraw.
        self.draw_canvas()

        # See if any bugs are still active.
        still_active = any(bug.is_active for bug in self.bugs)
        if not still_active:
            # No bugs are active. Stop the simulation.
            self.stop()
        else:
            # Run again later.
            self.window.after(TICK_MS, self.tick)

    # Map world to device coordinates.
    def xy_w_to_d(self, x, y):
        new_x = self.dxmin + (x - self.wxmin) * self.xscale
        new_y = self.dymin + (y - self.wymin) * self.yscale
        return Point2d(new_x, new_y)

    def point_w_to_d(self, point):
        return self.xy_w_to_d(point.x, point.y)

    # Draw the bugs.
    def draw_canvas(self):
        self.canvas.delete(tk.ALL)

        # Draw the checkerboard.
        self.draw_checkerboard(self.wxmin, self.wxmax, 1,
            self.wymin, self.wymax, 1)

        # Draw the bugs.
        for bug in self.bugs:
            self.draw_bug(bug)

        # Draw the global best point.
        self.draw_circle(Bug.global_best_location, 5, 'blue')

    # Draw the checkerboard.
    def draw_checkerboard(self, xmin, xmax, xstep, ymin, ymax, ystep):
        # Draw a checkerboard.
        x = xmin
        while x <= xmax:
            p0 = self.xy_w_to_d(x, ymin)
            p1 = self.xy_w_to_d(x, ymax)
            self.canvas.create_line(p0.x, p0.y, p1.x, p1.y, fill='gray')
            x += xstep
        y = ymin
        while y <= ymax:
            p0 = self.xy_w_to_d(ymin, y)
            p1 = self.xy_w_to_d(ymax, y)
            self.canvas.create_line(p0.x, p0.y, p1.x, p1.y, fill='gray')
            y += ystep

    # Draw a bug.
    def draw_bug(self, bug):
        if bug.is_active:
            self.draw_circle(bug.location, 2, 'orange')
        else:
            self.draw_circle(bug.location, 2, 'red')
        self.draw_circle(bug.best_location, 2, 'lime')

    # Draw a circle at the given point (in world coordinates)
    # with the given radius (in device coordinates).
    def draw_circle(self, point, radius, color):
        center = self.point_w_to_d(point)
        self.canvas.create_oval( \
            center.x - radius, center.y - radius, \
            center.x + radius, center.y + radius, \
            fill=color, outline=color)

if __name__ == '__main__':
    app = App()

invalid command name "4417614080tick"
    while executing
"4417614080tick"
    ("after" script)
